In [ ]:
'''
Example of tf-idf vectorizer.  Creates a "document" using each present cohort for each db.
'''

In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import tqdm
import numpy as np
from collections import OrderedDict
from collections import defaultdict
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import sys

In [4]:
sys.path.append('C:\\Users\\tonys\\Documents\\Research\\characterizationPaperPackage\\inst\\py\\')

In [5]:
from settings import *

Imported settings...
Current path:  C:\Users\tonys\Documents\Research\characterizationPaperPackage\inst\py\


In [15]:
summary_fp = '../../output/summaries/' + db + '/'

In [19]:
import glob

def dd():
    return defaultdict(int)

concept_id_mapper = defaultdict(dd) # dd is a module-level function
# concept_id_mapper = defaultdict(lambda: 'missing')

In [10]:
cohort_files = pd.read_csv('../settings/CohortsToCreateFinal.csv')

In [11]:
created = []
for summary_file in tqdm.tqdm(glob.glob(summary_fp + '*.csv')):
    cohort_id = summary_file.split('\\')[-1].split('_')[0]
    created.append(cohort_id)

100%|████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 113872.51it/s]


In [13]:
for x in cohort_files.cohortId:
    if str(x) not in created:
        print(x)

11152
11167
11191
11218
11219


In [48]:
print('Creating document setup...')

documents = []
cohort = []

for summary_file in tqdm.tqdm(glob.glob(summary_fp + '*.csv')):
    cohort_id = summary_file.split('\\')[-1].split('_')[0]
    # created.append(cohort_id)
    
    summary = pd.read_csv(summary_file)
    
    for x in zip([x for x in summary.condition_concept_id.values], [x for x in summary.concept_name.values]):
        concept_id_mapper[x[0]] = x[1]
    
    cohort.append(cohort_id)
    
    document_female = ''
    document_male   = ''

    for i, row in summary.iterrows():
        document_female = document_female + (str(row.condition_concept_id) + ' ') * row.num_females_this_symptom
        document_male = document_male + (str(row.condition_concept_id) + ' ') * row.num_males_this_symptom

    big_document = document_female + document_male
    big_document = big_document.strip()
    documents.append(big_document)

with open('../../output/created_files/' + db + '_common_symptoms.pickle', 'wb') as handle:
    pickle.dump(concept_id_mapper, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Calculating tf-idf...')

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(documents)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()

Creating document setup...


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:43<00:00,  2.61it/s]


Calculating tf-idf...


In [86]:
df = pd.DataFrame(denselist, columns=feature_names, index = cohort)

In [87]:
df.head()

,132258,132277,132333,132342,132344,132356,132391,132392,132393,132397,...,81931,81937,81939,81942,81943,81948,81984,81987,81988,81989
11111,0.0,0.001254,0.0,0.000177,0.001596,0.000000,0.000151,0.000000,0.000381,0.000933,...,0.001683,0.000000,0.0,0.002954,0.000000,0.000000,0.006101,0.000000,0.0,0.0
11112,0.0,0.000929,0.0,0.001146,0.001478,0.000000,0.000000,0.000000,0.007292,0.000000,...,0.001204,0.000000,0.0,0.000000,0.000000,0.000000,0.001726,0.000000,0.0,0.0
11113,0.0,0.001148,0.0,0.001491,0.001875,0.000000,0.000255,0.000000,0.007065,0.000882,...,0.001664,0.000000,0.0,0.001698,0.000231,0.000000,0.002199,0.000000,0.0,0.0
11114,0.0,0.001178,0.0,0.000275,0.001519,0.000151,0.000179,0.000000,0.001379,0.000796,...,0.001057,0.000000,0.0,0.001586,0.000000,0.000000,0.003005,0.000311,0.0,0.0
11115,0.0,0.001312,0.0,0.000607,0.001584,0.000000,0.000329,0.000145,0.003237,0.001294,...,0.001253,0.000103,0.0,0.001847,0.000089,0.000213,0.002267,0.000137,0.0,0.0


In [88]:
df.reset_index(level=0, inplace=True)
df = df.rename(columns ={'index' : 'cohort_id'})
# df.reset_index(inplace=True)
df.head()

,cohort_id,132258,132277,132333,132342,132344,132356,132391,132392,132393,...,81931,81937,81939,81942,81943,81948,81984,81987,81988,81989
0,11111,0.0,0.001254,0.0,0.000177,0.001596,0.000000,0.000151,0.000000,0.000381,...,0.001683,0.000000,0.0,0.002954,0.000000,0.000000,0.006101,0.000000,0.0,0.0
1,11112,0.0,0.000929,0.0,0.001146,0.001478,0.000000,0.000000,0.000000,0.007292,...,0.001204,0.000000,0.0,0.000000,0.000000,0.000000,0.001726,0.000000,0.0,0.0
2,11113,0.0,0.001148,0.0,0.001491,0.001875,0.000000,0.000255,0.000000,0.007065,...,0.001664,0.000000,0.0,0.001698,0.000231,0.000000,0.002199,0.000000,0.0,0.0
3,11114,0.0,0.001178,0.0,0.000275,0.001519,0.000151,0.000179,0.000000,0.001379,...,0.001057,0.000000,0.0,0.001586,0.000000,0.000000,0.003005,0.000311,0.0,0.0
4,11115,0.0,0.001312,0.0,0.000607,0.001584,0.000000,0.000329,0.000145,0.003237,...,0.001253,0.000103,0.0,0.001847,0.000089,0.000213,0.002267,0.000137,0.0,0.0


In [89]:
df.to_csv('../../output/created_files/' + db + '_common_symptom_tfidf.csv', index=True)

In [90]:
final = df.transpose()
cols = [int(x) for x in final.iloc[0].tolist()]
final.columns = cols
final.drop('cohort_id', axis=0, inplace=True)
final['concept_name'] = [concept_id_mapper[int(x)] for x in final.index]
final.reset_index(inplace=True)
final = final.rename(columns = {'index':'condition_concept_id'})
final.to_csv('../../output/created_files/' + db + '_common_symptom_tfidf_final.csv', index=False)

In [91]:
final.head()

,condition_concept_id,11111,11112,11113,11114,11115,11116,11117,11118,11119,...,11225,11226,11227,11228,11229,11230,11231,11232,11234,concept_name
0,132258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.001781,0.0,0.0,0.0,0.0,0.000157,0.0,Primary malignant neoplasm of frontal sinus
1,132277,0.001254,0.000929,0.001148,0.001178,0.001312,0.001842,0.001326,0.001312,0.0,...,0.0,0.0,0.0,0.000237,0.0,0.0,0.0,0.000796,0.000586,Neoplasm of uncertain behavior of endocrine gl...
2,132333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Acute perichondritis of pinna
3,132342,0.000177,0.001146,0.001491,0.000275,0.000607,0.000423,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000218,0.0,Apraxia
4,132344,0.001596,0.001478,0.001875,0.001519,0.001584,0.002419,0.001217,0.001252,0.001507,...,0.001864,0.001975,0.001394,0.000848,0.0,0.0,0.003758,0.000932,0.001397,Gingival and periodontal disease
